# Train and Predict Drones

# 🧠 1. Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

### 1.0 Global Definitions

In [1]:
TRAIN_PATH='./datasets/training'
VALID_PATH='./datasets/valid/'
TEST_PATH='./datasets/test/'

MODELS_PATH='models/'


### 1.1 Install dependencies

In [2]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.10.7-cp314-cp314-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached contourpy-1.3.3-cp314-cp314-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp314-cp314-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp314-cp314-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached numpy-2.2.6.tar.gz (20.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata 

### 1.2 Imports

In [2]:
from ultralytics.utils.plotting import plot_results
from ultralytics import YOLO
import ultralytics
import torch
ultralytics.checks()

Ultralytics 8.3.221 🚀 Python-3.9.25 torch-2.8.0+cu128 CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
Setup complete ✅ (8 CPUs, 23.3 GB RAM, 379.0/464.2 GB disk)


### 1.3 Version Info (optional)

In [3]:
import platform

print(f"Python version : {platform.python_version()}")
print(f"Ultralytics YOLO version : {ultralytics.__version__}")
print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device : {torch.cuda.get_device_name(0)}")
print(f"Operating system : {platform.system()} {platform.release()}")

Python version : 3.9.25
Ultralytics YOLO version : 8.3.221
PyTorch version : 2.8.0+cu128
CUDA available : False
Operating system : Linux 6.17.7-300.fc43.x86_64


# 📂 2. Dataset

### 2.1 Acquire Dataset

Download dataset from hugging face.

### 2.2 Data Structure Check

### 2.3 Dataset Configuration

# ⚙️ 3. Training

Purpose: Handle model setup and training configuration.

### 3.1 Model Selection

In [4]:
YOLO_MODELS = {
    "nano": "yolo11n.pt",
    "small": "yolo11s.pt",
    "medium": "yolo11m.pt",
    "large": "yolo11l.pt",
    "xlarge": "yolo11x.pt",
}

selected_size = "small"
model = YOLO(YOLO_MODELS[selected_size])

### 3.2 Training Configuration
Define hyperparameters (epochs, batch size, image size)

In [5]:
import yaml

data_yaml = dict(
    train = TRAIN_PATH,
    val = VALID_PATH,
    test= TEST_PATH,
    nc =1,
    names =['drone']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

%cat data.yaml

{names: [drone], nc: 1, test: ./datasets/test/, train: ./datasets/training, val: ./datasets/valid/}


In [6]:
train_config = {
    'data': 'data.yaml',
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'lr0': 0.01,
    'optimizer': 'SGD'
}

train_config

{'data': 'data.yaml',
 'epochs': 50,
 'imgsz': 640,
 'batch': 16,
 'lr0': 0.01,
 'optimizer': 'SGD'}

### 3.3 Run Training

In [8]:
results = model.train(**train_config)
plot_results("notebooks/computer_vision/results.csv")

New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.9.25 torch-2.8.0+cu128 CPU (11th Gen Intel Core i7-1165G7 @ 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, 

KeyboardInterrupt: 

### 3.4 Evaluate Model

In [ ]:
metrics = model.val()
metrics
metrics.confusion_matrix

# 🔍 4. Prediction & Visualization

Purpose: Evaluate results qualitatively and quantitatively.


### 4.1 Run Predictions

In [ ]:
preds = model.predict("test_images/", save=True)

### 4.2 Visualize Predictions

In [ ]:
from IPython.display import Image
Image(filename="runs/detect/predict/image_01.jpg")

# 📊 5. Analysis & Improvements
Purpose: Interpret and iterate.

### 5.1 Metrics Review

In [ ]:
metrics.box.map, metrics.box.map50, metrics.box.map75

### 5.2 Error Analysis

In [ ]:
metrics.confusion_matrix.plot()

# ✅ 6. Export & Deployment